In [2]:
!pip install -U \
    transformers \
    peft \
    datasets \
    torch==2.4.0 \
    torchvision==0.19.0 \
    accelerate \
    sentence-transformers==3.0.1 \
    faiss-cpu==1.8.0 \
    pandas==2.2.2 \
    tqdm \
    trl \
    bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 76.1 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 212.6 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 257.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 252.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 128.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 139.4 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 178.9 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 256.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 265.3 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 264.2 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 265.0 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [7]:
# Remove everything
!pip uninstall -y peft transformers accelerate bitsandbytes

# Install compatible versions
!pip install transformers==4.45.0 peft==0.13.0 accelerate==0.34.0 bitsandbytes==0.44.0

Found existing installation: peft 0.13.0
Uninstalling peft-0.13.0:
  Successfully uninstalled peft-0.13.0
Found existing installation: transformers 4.45.0
Uninstalling transformers-4.45.0:
  Successfully uninstalled transformers-4.45.0
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
  Using cached transformers-4.45.0-py3-none-any.whl.metadata (44 kB)
  Using cached peft-0.13.0-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.45.0-py3-none-any.whl (9.9 MB)
Using cached peft-0.13.0-py3-none-any.whl (322 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 113.3 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [peft]3/4 [peft]erate]s]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

## **fine tune model load from huggingface and inference now**

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, LoraConfig
import json
from huggingface_hub import hf_hub_download
import os

# Check versions
import transformers, peft
print(f"Transformers: {transformers.__version__}")
print(f"PEFT: {peft.__version__}")

# Model paths
model_name = "microsoft/Phi-3-mini-4k-instruct"
model_path = "alam1n/phi3-mbti-lora"

# Step 1: Download and fix the config file
print("Downloading and fixing adapter config...")
config_file = hf_hub_download(repo_id=model_path, filename="adapter_config.json")

with open(config_file, 'r') as f:
    config_data = json.load(f)

print(f"Original config keys: {list(config_data.keys())}")

# Create clean config - keep only what PEFT 0.13.0 understands
clean_config = {
    "base_model_name_or_path": config_data.get("base_model_name_or_path", model_name),
    "bias": config_data.get("bias", "none"),
    "fan_in_fan_out": config_data.get("fan_in_fan_out", False),
    "inference_mode": True,
    "init_lora_weights": config_data.get("init_lora_weights", True),
    "lora_alpha": config_data.get("lora_alpha", 32),
    "lora_dropout": config_data.get("lora_dropout", 0.05),
    "modules_to_save": config_data.get("modules_to_save"),
    "peft_type": "LORA",
    "r": config_data.get("r", 16),
    "target_modules": config_data.get("target_modules", []),
    "task_type": config_data.get("task_type", "CAUSAL_LM")
}

# Overwrite the config file
with open(config_file, 'w') as f:
    json.dump(clean_config, f, indent=2)

print("Config fixed!")

# Step 2: Load model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, model_path)

# Step 3: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Model loaded successfully!")

# Get special tokens
end_token_id = tokenizer.convert_tokens_to_ids(["<|end|>"])[0]

# Inference function
def predict_mbti(person_text):
    model.eval()
    prompt = f"""<|system|>
You are an expert in MBTI personality analysis. Analyze the person's writing and behavior to determine their exact 4-letter MBTI type (e.g., INTJ, ESFP). Return ONLY the type.<|end|>
<|user|>
Analyze this person's posts and determine their MBTI type:

"{person_text}"<|end|>
<|assistant|>
"""
    
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **input_ids,
            max_new_tokens=10,
            do_sample=False,
            eos_token_id=end_token_id,
            pad_token_id=end_token_id
        )
    
    generated_text = tokenizer.decode(outputs[:, input_ids["input_ids"].shape[-1]:][0], skip_special_tokens=False)
    return generated_text.split("<|end|>")[0].strip()

# Test
print("\n" + "="*60)
print("Testing prediction...")
print("="*60)
result = predict_mbti("I love analyzing complex systems and finding elegant solutions. Spent the weekend refactoring code.")
print(f"\n🎯 MBTI Type: {result}")

Transformers: 4.45.0
PEFT: 0.13.0
Original config keys: ['alpha_pattern', 'auto_mapping', 'base_model_name_or_path', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'inference_mode', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'peft_type', 'qalora_group_size', 'r', 'rank_pattern', 'revision', 'target_modules', 'target_parameters', 'task_type', 'trainable_token_indices', 'use_dora', 'use_qalora', 'use_rslora']
Config fixed!
Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LoRA adapter...


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


✅ Model loaded successfully!

Testing prediction...

🎯 MBTI Type: INTJ


## **Real data output Test**

In [3]:
person_1 = """
I spend most of my time analyzing complex systems and optimizing architectures.
Love diving deep into technical documentation and building efficient solutions.
Not big on meetings unless they're focused and productive. Prefer written communication.
Currently working on a multi-threading optimization project. The challenge is fascinating.
Weekend plans? Probably reading research papers on distributed systems."""

result = predict_mbti(person_1)
print(result)

INTP


In [4]:
person_2 = """
Just had the most amazing brainstorming session with the team! So many creative ideas!
Love connecting with people and exploring new possibilities. Every project is an adventure!
Can't wait to share our new product concept at the conference next week.
Also organizing a team outing - thinking escape room or something interactive and fun!
Life is too short to not follow your passions and inspire others along the way!"""

result = predict_mbti(person_2)
print(result)

ENFP


In [5]:
person_3 = """
Completed the quarterly compliance audit ahead of schedule. All documentation filed properly.
I believe in doing things right the first time. Follow procedures, meet deadlines, deliver quality.
Updated our standard operating procedures based on last month's review.
Maintaining our project tracking system and ensuring everyone follows the workflow.
Consistency and attention to detail are key to success in any organization.
"""

result = predict_mbti(person_3)
print(result)

ENTP


In [6]:
person_4 = """
Today was AMAZING! Closed three deals before lunch and celebrated with the team!
Love the energy of working with people - every client interaction is unique and exciting!
Hosting our department's happy hour this Friday. Already planning some fun activities!
Life's about living in the moment and making every day count. Work hard, play harder!
Just booked tickets for a spontaneous weekend trip. Who's coming with me?!
"""

result = predict_mbti(person_4)
print(result)

ENFP


In [7]:
person_5 = """
Been thinking about how our work impacts people's lives and how we can do more good.
Had a deep conversation with a colleague about creating meaningful change in our industry.
Working on a mentorship program to help junior team members find their purpose.
I believe technology should serve humanity and create a better future for everyone.
Sometimes I need quiet time to reflect and recharge, but I'm always thinking of others.
"""

result = predict_mbti(person_5)
print(result)

ENFP


In [8]:
# Test Case 1: INTJ - Strategic Analyst
person_1 = """
I spend most of my time analyzing complex systems and optimizing architectures.
Love diving deep into technical documentation and building efficient solutions.
Not big on meetings unless they're focused and productive. Prefer written communication.
Currently working on a multi-threading optimization project. The challenge is fascinating.
Weekend plans? Probably reading research papers on distributed systems.
"""

# Test Case 2: ENFP - Enthusiastic Communicator
person_2 = """
Just had the most amazing brainstorming session with the team! So many creative ideas!
Love connecting with people and exploring new possibilities. Every project is an adventure!
Can't wait to share our new product concept at the conference next week.
Also organizing a team outing - thinking escape room or something interactive and fun!
Life is too short to not follow your passions and inspire others along the way!
"""

# Test Case 3: ISTJ - Reliable Organizer
person_3 = """
Completed the quarterly compliance audit ahead of schedule. All documentation filed properly.
I believe in doing things right the first time. Follow procedures, meet deadlines, deliver quality.
Updated our standard operating procedures based on last month's review.
Maintaining our project tracking system and ensuring everyone follows the workflow.
Consistency and attention to detail are key to success in any organization.
"""

# Test Case 4: ESFP - Energetic Performer
person_4 = """
Today was AMAZING! Closed three deals before lunch and celebrated with the team!
Love the energy of working with people - every client interaction is unique and exciting!
Hosting our department's happy hour this Friday. Already planning some fun activities!
Life's about living in the moment and making every day count. Work hard, play harder!
Just booked tickets for a spontaneous weekend trip. Who's coming with me?!
"""

# Test Case 5: INFJ - Insightful Idealist
person_5 = """
Been thinking about how our work impacts people's lives and how we can do more good.
Had a deep conversation with a colleague about creating meaningful change in our industry.
Working on a mentorship program to help junior team members find their purpose.
I believe technology should serve humanity and create a better future for everyone.
Sometimes I need quiet time to reflect and recharge, but I'm always thinking of others.
"""

# Test Case 6: ENTP - Innovative Debater
person_6 = """
Just challenged our entire approach in the strategy meeting. Got everyone thinking differently!
Why do it the conventional way when we can innovate and disrupt the whole process?
Spent the evening exploring three different solutions to the same problem. All viable!
Love a good intellectual debate - it sharpens ideas and reveals better alternatives.
Starting a side project that combines AI with blockchain. High risk, high reward!
"""

# Test Case 7: ISFJ - Supportive Helper
person_7 = """
Made sure everyone on the team had what they needed for the presentation today.
I find satisfaction in supporting others and making sure things run smoothly.
Organized the welcome kit for our new hires - want them to feel comfortable here.
Following up on customer feedback to ensure we're meeting their expectations.
Small gestures matter. Brought coffee for the team working late on the project.
"""

# Test Case 8: ESTP - Action-Oriented Problem Solver
person_8 = """
Crisis in production? I'm on it. Fixed the server issue in 20 minutes while everyone panicked.
Don't overthink it - take action, solve problems, move forward. That's my motto.
Negotiated a great deal with the vendor this morning. Quick thinking pays off!
Leading the emergency response team. We handle pressure and deliver results fast.
Life's too short for endless planning. Sometimes you just gotta do it and figure it out!
"""

# Run all tests
test_cases = [
    ("Strategic Analyst", person_1),
    ("Enthusiastic Communicator", person_2),
    ("Reliable Organizer", person_3),
    ("Energetic Performer", person_4),
    ("Insightful Idealist", person_5),
    ("Innovative Debater", person_6),
    ("Supportive Helper", person_7),
    ("Action-Oriented Problem Solver", person_8)
]

print("\n" + "="*70)
print("MBTI PREDICTION TEST SUITE")
print("="*70)

for name, text in test_cases:
    result = predict_mbti(text)
    print(f"\n🧑 {name}")
    print(f"🎯 Predicted Type: {result}")
    print("-" * 70)

# Interactive mode - paste your own text
print("\n" + "="*70)
print("INTERACTIVE MODE")
print("="*70)
print("\nPaste your text below (or type 'exit' to quit):\n")

while True:
    user_input = input("Your text: ").strip()
    if user_input.lower() in ['exit', 'quit', '']:
        print("Goodbye!")
        break
    
    result = predict_mbti(user_input)
    print(f"🎯 Predicted MBTI Type: {result}\n")


MBTI PREDICTION TEST SUITE

🧑 Strategic Analyst
🎯 Predicted Type: INTP
----------------------------------------------------------------------

🧑 Enthusiastic Communicator
🎯 Predicted Type: ENFP
----------------------------------------------------------------------

🧑 Reliable Organizer
🎯 Predicted Type: ENTP
----------------------------------------------------------------------

🧑 Energetic Performer
🎯 Predicted Type: ENFP
----------------------------------------------------------------------

🧑 Insightful Idealist
🎯 Predicted Type: ENFP
----------------------------------------------------------------------

🧑 Innovative Debater
🎯 Predicted Type: ENTP
----------------------------------------------------------------------

🧑 Supportive Helper
🎯 Predicted Type: ENTP
----------------------------------------------------------------------

🧑 Action-Oriented Problem Solver
🎯 Predicted Type: ENTP
----------------------------------------------------------------------

INTERACTIVE MODE

Paste 

Your text:  exit


Goodbye!


## Expected output Test

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# --------------------------------------------------------------
# 1. Load model + LoRA (4-bit)
# --------------------------------------------------------------
model_name = "microsoft/Phi-3-mini-4k-instruct"
adapter_name = "alam1n/phi3-mbti-lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------------------------------------------------------------
# 2. MBTI Table (16 types)
# --------------------------------------------------------------
MBTI_TABLE = {
    "INTJ": ("Strategic, independent", "Original minds with drive for goals", "Strong in planning roles, e.g., tech startups"),
    "INTP": ("Analytical, inventive", "Seek logical explanations", "Great for R&D in Dhaka AI labs"),
    "ENTJ": ("Decisive, leader", "Assume leadership readily", "Ideal for scaling startups"),
    "ENTP": ("Innovative, debating", "Resourceful in new challenges", "Perfect for product ideation"),
    "INFJ": ("Insightful, idealistic", "Seek meaning & connection", "Excellent in HR / culture building"),
    "INFP": ("Creative, empathetic", "Loyal to values", "Strong in content & community"),
    "ENFJ": ("Charismatic, mentoring", "Attuned to others' emotions", "Great for sales & partnerships"),
    "ENFP": ("Enthusiastic, imaginative", "See possibilities", "Best for marketing & outreach"),
    "ISTJ": ("Dependable, thorough", "Responsible & detail-oriented", "Core for operations & compliance"),
    "ISFJ": ("Conscientious, supportive", "Committed to obligations", "Perfect for customer success"),
    "ESTJ": ("Organized, decisive", "Implement decisions fast", "Strong in project management"),
    "ESFJ": ("Harmonious, cooperative", "Want harmony", "Ideal for team coordination"),
    "ISTP": ("Practical, adaptable", "Quick problem-solver", "Excellent in DevOps / troubleshooting"),
    "ISFP": ("Artistic, flexible", "Enjoy present moment", "Great for UI/UX design"),
    "ESTP": ("Action-oriented, pragmatic", "Focus on immediate results", "Perfect for field sales"),
    "ESFP": ("Outgoing, spontaneous", "Enjoy working with people", "Best for events & community"),
}

# --------------------------------------------------------------
# 3. Correct inference function (EXACT prompt used in training)
# --------------------------------------------------------------
# --------------------------------------------------------------
# 3. Inference – robust cleaning + table output
# --------------------------------------------------------------
import re

def predict_mbti_and_table(person_text: str) -> str:
    if len(person_text) > 3200:
        person_text = person_text[:3200] + "..."

    prompt = f"""<|system|>
You are an MBTI expert. Return ONLY the 4-letter type.<|end|>
<|user|>
Analyze:

"{person_text}"<|end|>
<|assistant|>"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,
            temperature=None,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    reply = tokenizer.decode(output[0], skip_special_tokens=True)
    print("\n--- RAW REPLY ---")
    print(reply)
    print("--- END RAW ---\n")

    # ✅ NEW: Extract MBTI using regex
    match = re.findall(r'\b[IE][NS][FT][JP]\b', reply.upper())
    mbti = match[-1] if match else "UNKNOWN"

    traits, desc, context = MBTI_TABLE.get(mbti, ("–", "–", "–"))

    table = f"""
| MBTI Type | Key Traits            | Description                         | Fit for Dhaka Business Context                     |
|-----------|-----------------------|-------------------------------------|----------------------------------------------------|
| **{mbti}**    | {traits}             | {desc}             | {context}                                         |
"""
    return table.strip()


# --------------------------------------------------------------
# 4. Test with real data
# --------------------------------------------------------------
person_text = """
Name: Al Amin
Headline: Senior Software Engineer | AI Enthusiast
Location: Dhaka, Bangladesh
About: Passionate about building scalable systems and mentoring juniors. Love diving deep into algorithms and optimizing performance.
Posts: Just deployed a new RAG pipeline — 40% faster retrieval! | Teaching a workshop on LoRA fine-tuning next week. Excited to share knowledge! | Spent weekend reading papers on efficient transformers.
"""

print("Testing prediction...")
print("="*70)
print(predict_mbti_and_table(person_text))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Testing prediction...

--- RAW REPLY ---
You are an MBTI expert. Return ONLY the 4-letter type. Analyze:

"
Name: Al Amin
Headline: Senior Software Engineer | AI Enthusiast
Location: Dhaka, Bangladesh
About: Passionate about building scalable systems and mentoring juniors. Love diving deep into algorithms and optimizing performance.
Posts: Just deployed a new RAG pipeline — 40% faster retrieval! | Teaching a workshop on LoRA fine-tuning next week. Excited to share knowledge! | Spent weekend reading papers on efficient transformers.
" ENFJ
--- END RAW ---

| MBTI Type | Key Traits            | Description                         | Fit for Dhaka Business Context                     |
|-----------|-----------------------|-------------------------------------|----------------------------------------------------|
| **ENFJ**    | Charismatic, mentoring             | Attuned to others' emotions             | Great for sales & partnerships                                         |
